# Orbita2d Python API example

## Usage

Assuming you have already built the bindings in your virtual environment, you can directly import the `orbita2d` module.

If it's not already done, you can first run ```maturin develop``` from the orbita2d_c_api folder.

In [1]:
import orbita2d

Or to directly import a controller:

In [2]:
from orbita2d import Orbita2dController

You can then connect to the actuator using the `from_config` method:

Configuration examples can be found here: [orbita2d_controller/config](https://github.com/pollen-robotics/orbita2d_control/tree/develop/orbita2d_controller/config)

In [3]:
configfile = "/home/steve/Project/Repo/Pollen/orbita2d_control/orbita2d_controller/config/right_elbow_flipsky_2.yaml"

orbita2d_controller = Orbita2dController.from_config(configfile)

## Control your Orbita2d

### Enable/disable torque

To enable or disable the torque you can use the following methods:

To enable the torque (note the _reset_target_ parameter that is used to reset the target position to the current position and make sure your actuator won't go to a previously set target position):

In [4]:
orbita2d_controller.enable_torque(reset_target=True)

And to disable it:

In [5]:
orbita2d_controller.disable_torque()

You can also check if the torque is enabled:

In [ ]:
orbita2d_controller.is_torque_on()

## Read and set positions

To read the current position of the actuator, you can use the `get_current_orientation` method:

In [ ]:
ring, center = orbita2d_controller.get_current_orientation()
print(f'Orbita current orientation: ring={ring}rads, center={center}rads')

And to set a target position, you can use the `set_target_orientation` method (make sure to enable the torque first and that the actuator can freely move to the target position):

In [ ]:
import time

orbita2d_controller.enable_torque(reset_target=True)

orbita2d_controller.set_target_orientation(target=(0.0, 0.0))
time.sleep(1)
orbita2d_controller.set_target_orientation(target=(0.0, 0.5))
time.sleep(1)
orbita2d_controller.set_target_orientation(target=(0.5, 0.5))
time.sleep(1)
orbita2d_controller.set_target_orientation(target=(0.5, 0.0))
time.sleep(1)
orbita2d_controller.set_target_orientation(target=(0.0, 0.0))

You can also play a more complex trajectory and records the real reached positions. Let's play a sinusoid trajectory on the center axis:

In [ ]:
import numpy as np
import time

duration = 10
amp = np.deg2rad(20)
freq = 0.5

recorded_pos = []

t0 = time.time()
while time.time() - t0 < duration:
    center_target = amp * np.sin(2 * np.pi * freq * (time.time() - t0))

    # Set the target orientation    
    orbita2d_controller.set_target_orientation(target=(0.0, center_target))

    # Record the current orientation
    recorded_pos.append(orbita2d_controller.get_current_orientation())

    time.sleep(0.001)

Using matplotlib, we can plot the real reached positions:

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(recorded_pos)

Similarly, you can read the current velocity of the actuator using the `get_current_velocity` method. 

You can also read the current torque using the `get_current_torque` method.

## Extra parameters

You can also access (get/set) the following parameters:
* velocity_limit: `get_raw_motors_velocity_limit`, `set_raw_motors_velocity_limit`
* torque_limit: `get_raw_motors_torque_limit`, `set_raw_motors_torque_limit`
* pid_gains: `get_raw_motors_pid_gains`, `set_raw_motors_pid_gains`


**Be careful when setting these parameters, as they control directly the motors used within Orbita2d not the actuator itself!**

## Kinematics

The Python API also provides access to the kinematics of the actuator. You can compute forward/inverse kinematics of position/velocity/torque. All angles are given in radians.

First, creates your kinematics model based on the motor ratios of your actuator. If you are not sure, they can be found in the configuration file you are using.

In [1]:
from orbita2d import KinematicsModel

kinematics = KinematicsModel(ratio_a=47.519, ratio_b=47.519)

For the position:

In [6]:
ring, center = kinematics.forward_position(angle_a=0.0, angle_b=10.0)
(ring, center)

(0.10522106946695006, -0.10522106946695006)

In [7]:
kinematics.inverse_position(ring, center)

(0.0, 10.0)